Installs:

In [ ]:
!pip install -q segmentation-models-pytorch albumentations monai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 81.1 MB/s eta 0:00:00


Imports:

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
from tqdm import tqdm
import random

# Import advanced loss functions
from monai.losses import DiceLoss, HausdorffDTLoss

<frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.


In [ ]:
class Config:
    # -- Base Paths --
    # Points to the new COMBINED dataset (Original + Synthetic)
    BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/phenocyte_seg/phenocyte_seg/"
    DATA_PATH = os.path.join(BASE_PATH, "combined_data")

    IMAGE_DIR = os.path.join(DATA_PATH, "images")
    MASK_DIR = os.path.join(DATA_PATH, "masks")

    # Path to the Quality Control CSV
    QC_REPORT_CSV = os.path.join(BASE_PATH, "missing_classes_from_mask.csv")

    # -- Output Paths --
    OUTPUT_DIR = os.path.join(BASE_PATH, "outputs_v3") # V3 Output
    OUTPUT_MASK_DIR = os.path.join(OUTPUT_DIR, "pred_masks_unsqueezed")
    COLOR_MASK_DIR = os.path.join(OUTPUT_DIR, "color_masks_unsqueezed")

    # -- Data Split Sizes --
    # Since we are filtering data, the total available images might change (diff of 49 images).
    VAL_SIZE = 51
    TEST_SIZE = 52
    # Train size will be whatever is left

    # -- Model Hyperparameters --
    ARCHITECTURE = 'unet'
    ENCODER = 'resnet34'
    ENCODER_WEIGHTS = 'imagenet'
    LEARNING_RATE = 1e-4
    OPTIMIZER = 'AdamW'

    # -- Training Settings --
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    BATCH_SIZE = 4
    NUM_EPOCHS = 30
    IMAGE_HEIGHT = 256
    IMAGE_WIDTH = 256
    NUM_CLASSES = 5

    # -- LOSS CONFIGURATION (Successful V2 Settings) --
    # Weights: [Background, Stem, Leaf, Root, Seed]
    # We use the moderate weights that worked well
    CLASS_WEIGHTS = torch.tensor([
        1.0,  # Background
        7.0,  # Stem
        5.0,  # Leaf
        5.0,  # Root
        7.0   # Seed
    ], device=DEVICE)

    # Weights for the hybrid loss (Pure Weighted Dice)
    HYBRID_WEIGHT_DICE = 1.0
    HYBRID_WEIGHT_HD = 0.0

    # -- Visualization --
    COLOR_MAP = {
        0: (0, 0, 0),         # background - black
        1: (139, 69, 19),     # stem - brown
        2: (0, 255, 0),       # leaf - green
        3: (255, 255, 0),     # root - yellow
        4: (255, 0, 0),       # seed - red
    }

os.makedirs(Config.OUTPUT_MASK_DIR, exist_ok=True)
os.makedirs(Config.COLOR_MASK_DIR, exist_ok=True)

Loads all images, excludes those marked FALSE in the CSV, and splits the rest.

In [ ]:
def get_filtered_data_splits(image_dir, qc_csv_path, val_size, test_size):
    # 1. Get all available images in the combined folder
    all_images = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))])
    print(f"Total images found in folder: {len(all_images)}")

    # 2. Load the unsafe image list
    excluded_filenames = set()
    if os.path.exists(qc_csv_path):
        df = pd.read_csv(qc_csv_path)
        # We look for rows where Mask_correct is FALSE (boolean or string)
        bad_rows = df[df['Mask_correct'].astype(str).str.upper() == 'FALSE']
        excluded_filenames = set(bad_rows['filename'].tolist())
        print(f"Found {len(excluded_filenames)} images marked as FALSE in CSV to exclude.")
    else:
        print("Warning:CSV not found. Proceeding without filtering.")

    # 3. Filter the list
    valid_images = []
    for img_name in all_images:
        # We need to check the MASK name against the blacklist
        # Assuming standard naming: image.jpg -> image_mask.png
        mask_name = os.path.splitext(img_name)[0] + "_mask.png"

        if mask_name in excluded_filenames:
            continue # Skip this image

        valid_images.append(img_name)

    print(f"Total Valid Images for Training: {len(valid_images)}")

    # 4. Shuffle and Split
    random.seed(42)
    random.shuffle(valid_images)

    # Prioritize Test and Val sets, give rest to Train
    if len(valid_images) < (val_size + test_size):
        raise ValueError("Not enough valid images to create Validation and Test sets!")

    test_files = valid_images[:test_size]
    val_files = valid_images[test_size : test_size + val_size]
    train_files = valid_images[test_size + val_size :]

    print(f"Training set size: {len(train_files)}")
    print(f"Validation set size: {len(val_files)}")
    print(f"Test set size: {len(test_files)}")

    return train_files, val_files, test_files

In [ ]:
class PlantDataset(Dataset):
    def __init__(self, image_dir, mask_dir, image_filenames, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = image_filenames

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_name = self.images[index]
        img_path = os.path.join(self.image_dir, img_name)
        mask_name = os.path.splitext(img_name)[0] + "_mask.png"
        mask_path = os.path.join(self.mask_dir, mask_name)

        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)

        # Get original dimensions for unsqueezing
        original_height, original_width = image.shape[:2]

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        # Clip mask to be safe (0 to 4)
        mask[mask >= Config.NUM_CLASSES] = 0

        return image, mask.long(), (original_height, original_width)

# Augmentations
train_transform = A.Compose([
    A.Resize(height=Config.IMAGE_HEIGHT, width=Config.IMAGE_WIDTH),
    A.Rotate(limit=35, p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0], max_pixel_value=255.0),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(height=Config.IMAGE_HEIGHT, width=Config.IMAGE_WIDTH),
    A.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0], max_pixel_value=255.0),
    ToTensorV2(),
])

In [ ]:
class WeightedHybridLoss(nn.Module):
    def __init__(self, class_weights, weight_dice=1.0, weight_hd=0.0):
        super(WeightedHybridLoss, self).__init__()
        self.w_dice = weight_dice
        self.w_hd = weight_hd
        self.class_weights = class_weights

        self.dice_loss = DiceLoss(
            softmax=True, to_onehot_y=True, include_background=True, reduction='none'
        )
        self.hd_loss = HausdorffDTLoss(
            softmax=True, to_onehot_y=True, include_background=True, reduction='none'
        )

    def forward(self, preds_logits, targets_idx):
        targets_idx = targets_idx.unsqueeze(1)

        loss_dice_per_class = self.dice_loss(preds_logits, targets_idx)

        # Apply weights to Dice
        weighted_loss_dice_all = loss_dice_per_class * self.class_weights
        weighted_loss_dice = weighted_loss_dice_all.mean()

        total_loss = self.w_dice * weighted_loss_dice

        # Add HD loss if weight > 0 (Currently 0 in Config)
        if self.w_hd > 0:
            loss_hd_per_class = self.hd_loss(preds_logits, targets_idx)
            weighted_loss_hd_all = loss_hd_per_class * self.class_weights
            weighted_loss_hd = weighted_loss_hd_all.mean()
            total_loss += (self.w_hd * weighted_loss_hd)

        return total_loss

In [ ]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader, desc="Training")
    total_loss = 0

    for batch_idx, (data, targets, _) in enumerate(loop):
        data = data.to(device=Config.DEVICE)
        targets = targets.to(device=Config.DEVICE)

        with torch.amp.autocast('cuda'):
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    return total_loss / len(loader)

In [ ]:
def eval_fn(loader, model, loss_fn):
    model.eval()
    total_loss = 0
    loop = tqdm(loader, desc="Validation")

    with torch.no_grad():
        for data, targets, _ in loop:
            data = data.to(device=Config.DEVICE)
            targets = targets.to(device=Config.DEVICE)
            predictions = model(data)
            loss = loss_fn(predictions, targets)
            total_loss += loss.item()
            loop.set_postfix(val_loss=loss.item())

    model.train()
    return total_loss / len(loader)

In [ ]:
def mask_to_rgb(mask_tensor, color_map):
    mask = mask_tensor.cpu().numpy().squeeze()
    rgb_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
    for class_idx, color in color_map.items():
        rgb_mask[mask == class_idx] = color
    return Image.fromarray(rgb_mask)

In [ ]:
def save_predictions_fn(loader, model, folder_basename=""):
    print(f"\n--- Saving predictions for {folder_basename} set ---")
    model.eval()

    # Define sub-folders for output
    output_mask_dir = os.path.join(Config.OUTPUT_MASK_DIR, folder_basename)
    color_mask_dir = os.path.join(Config.COLOR_MASK_DIR, folder_basename)
    os.makedirs(output_mask_dir, exist_ok=True)
    os.makedirs(color_mask_dir, exist_ok=True)

    for idx in tqdm(range(len(loader.dataset)), desc=f"Saving {folder_basename} Predictions"):
        img_tensor, _, (original_h, original_w) = loader.dataset[idx]

        with torch.no_grad():
            img_tensor = img_tensor.to(Config.DEVICE).unsqueeze(0)
            preds = model(img_tensor)
            final_mask_tensor = torch.argmax(preds, dim=1).squeeze(0)

        pred_mask_np = final_mask_tensor.cpu().numpy().astype(np.uint8)

        # Unsqueeze
        resized_mask = cv2.resize(
            pred_mask_np,
            (original_w, original_h),
            interpolation=cv2.INTER_NEAREST
        )

        # Save Raw Mask
        pred_mask_img = Image.fromarray(resized_mask)
        original_filename = loader.dataset.images[idx]
        mask_filename = os.path.splitext(original_filename)[0] + "_mask.png"
        pred_mask_img.save(os.path.join(output_mask_dir, mask_filename))

        # Save Color Mask
        color_mask_img = mask_to_rgb(torch.from_numpy(resized_mask), Config.COLOR_MAP)
        color_mask_img.save(os.path.join(color_mask_dir, mask_filename))

    model.train()

In [ ]:
def main():
    print(f"Using device: {Config.DEVICE}")
    print(f"Dataset Path: {Config.DATA_PATH}")

    # --- 1. Get Filtered Splits ---
    train_files, val_files, test_files = get_filtered_data_splits(
        Config.IMAGE_DIR, Config.QC_REPORT_CSV, Config.VAL_SIZE, Config.TEST_SIZE
    )

    # --- 2. Create Loaders ---
    train_dataset = PlantDataset(Config.IMAGE_DIR, Config.MASK_DIR, train_files, train_transform)
    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True)

    val_dataset = PlantDataset(Config.IMAGE_DIR, Config.MASK_DIR, val_files, val_transform)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE, shuffle=False)

    test_dataset = PlantDataset(Config.IMAGE_DIR, Config.MASK_DIR, test_files, val_transform)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    # --- 3. Model & Loss ---
    model = smp.create_model(
        arch=Config.ARCHITECTURE,
        encoder_name=Config.ENCODER,
        encoder_weights=Config.ENCODER_WEIGHTS,
        in_channels=3,
        classes=Config.NUM_CLASSES,
    ).to(Config.DEVICE)

    print("\n--- Initializing V3 Loss Function ---")
    loss_fn = WeightedHybridLoss(
        class_weights=Config.CLASS_WEIGHTS,
        weight_dice=Config.HYBRID_WEIGHT_DICE,
        weight_hd=Config.HYBRID_WEIGHT_HD
    )
    print(f"Class Weights: {Config.CLASS_WEIGHTS.cpu().numpy()}")

    optimizer = torch.optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE)
    scaler = torch.amp.GradScaler('cuda')
    best_val_loss = float('inf')

    # --- 4. Training Loop ---
    for epoch in range(Config.NUM_EPOCHS):
        print(f"\n--- Epoch {epoch+1}/{Config.NUM_EPOCHS} ---")
        train_loss = train_fn(train_loader, model, optimizer, loss_fn, scaler)
        val_loss = eval_fn(val_loader, model, loss_fn)

        print(f"Average Train Loss: {train_loss:.4f}")
        print(f"Average Val Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), os.path.join(Config.BASE_PATH, "best_model_v3.pth"))
            print("=> Saved new best model")

    # --- 5. Save Test Predictions ---
    print("\n--- Loading best model for final testing ---")
    model.load_state_dict(torch.load(os.path.join(Config.BASE_PATH, "best_model_v3.pth")))
    save_predictions_fn(test_loader, model, folder_basename="test_set")
    print("--- V3 Training Complete ---")

In [ ]:
if __name__ == "__main__":
    main()

Using device: cuda
Dataset Path: /content/drive/MyDrive/Colab Notebooks/phenocyte_seg/phenocyte_seg/combined_data
Total images found in folder: 595
Found 32 images marked as FALSE in CSV to exclude.
Total Valid Images for Training: 563
Training set size: 460
Validation set size: 51
Test set size: 52


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]


--- Initializing V3 Loss Function ---
Class Weights: [1. 7. 5. 5. 7.]

--- Epoch 1/30 ---


...

Average Train Loss: 1.1011
Average Val Loss: 1.0977

--- Epoch 30/30 ---


Validation: 100%|██████████| 13/13 [00:00<00:00, 25.32it/s, val_loss=0.872]


Average Train Loss: 1.0881
Average Val Loss: 1.0662
=> Saved new best model

--- Loading best model for final testing ---

--- Saving predictions for test_set set ---


Saving test_set Predictions: 100%|██████████| 52/52 [00:50<00:00,  1.02it/s]

--- V3 Training Complete ---


In [ ]:
# =================================================================================
# 0. SETUP AND IMPORTS
# =================================================================================
!pip install -q monai pandas

import os
import cv2
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from monai.metrics import (
    compute_dice,
    compute_iou,
    compute_hausdorff_distance
)

# =================================================================================
# 1. CONFIGURATION
# =================================================================================
# --- PATHS FOR V3 ---
# Ground Truth Masks (The COMBINED dataset masks)
GT_MASK_DIR = "/content/drive/MyDrive/Colab Notebooks/phenocyte_seg/phenocyte_seg/combined_data/masks/"

# Predicted Masks (From V3 Output)
PRED_MASK_DIR = "/content/drive/MyDrive/Colab Notebooks/phenocyte_seg/phenocyte_seg/outputs_v3/pred_masks_unsqueezed/test_set/"

# --- CORRECTED CLASS MAP ---
NUM_CLASSES = 5
CLASS_MAP = {
    0: "Background",
    1: "Stem",
    2: "Leaf",
    3: "Root",
    4: "Seed",
}

# =================================================================================
# 2. HELPER FUNCTIONS
# =================================================================================
def to_one_hot(mask, num_classes):
    """Converts a (H, W) mask to (1, C, H, W) one-hot tensor."""
    # Clip values just in case
    mask[mask >= num_classes] = 0

    one_hot = np.eye(num_classes)[mask] # (H, W, C)
    one_hot = np.transpose(one_hot, (2, 0, 1)) # (C, H, W)
    return torch.from_numpy(one_hot).unsqueeze(0) # (1, C, H, W)

# =================================================================================
# 3. MAIN ANALYSIS LOOP
# =================================================================================
def run_analysis():
    print("Starting V3 Analysis...")
    print(f"GT Directory: {GT_MASK_DIR}")
    print(f"Pred Directory: {PRED_MASK_DIR}")

    results_list = []
    pred_files = [f for f in os.listdir(PRED_MASK_DIR) if f.endswith('.png')]

    if len(pred_files) == 0:
        print("Error: No prediction files found! Check your PRED_MASK_DIR path.")
        return

    for filename in tqdm(pred_files):
        pred_path = os.path.join(PRED_MASK_DIR, filename)

        # Logic to find GT file (Assumes names match exactly or close to it)
        # V3 script saves predictions as "original_name_mask.png"
        gt_path = os.path.join(GT_MASK_DIR, filename)

        if not os.path.exists(gt_path):
            print(f"Skipping {filename}: GT mask not found.")
            continue

        # Load Masks
        gt_mask = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
        pred_mask = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)

        if gt_mask is None or pred_mask is None:
            continue

        # Resize GT if dimensions don't match (Safety check for unsqueezing issues)
        if gt_mask.shape != pred_mask.shape:
            # We assume prediction is correct size (unsqueezed), so we resize GT to match
            # This handles cases where GT might be slightly off due to pre-processing
            gt_mask = cv2.resize(gt_mask, (pred_mask.shape[1], pred_mask.shape[0]), interpolation=cv2.INTER_NEAREST)

        # Convert to One-Hot
        gt_onehot = to_one_hot(gt_mask, NUM_CLASSES)
        pred_onehot = to_one_hot(pred_mask, NUM_CLASSES)

        # --- Calculate Metrics ---
        dice = compute_dice(pred_onehot, gt_onehot, include_background=True)
        iou = compute_iou(pred_onehot, gt_onehot, include_background=True)
        hd95 = compute_hausdorff_distance(pred_onehot, gt_onehot, include_background=True, percentile=95)

        # Store results
        file_metrics = {'filename': filename}
        for i in range(NUM_CLASSES):
            c_name = CLASS_MAP[i]
            file_metrics[f"{c_name}_Dice"] = dice[0, i].item()
            file_metrics[f"{c_name}_IOU"] = iou[0, i].item()
            file_metrics[f"{c_name}_HD95"] = hd95[0, i].item()

        results_list.append(file_metrics)

    # =================================================================================
    # 4. REPORTING
    # =================================================================================
    if not results_list:
        print("No results generated.")
        return

    df = pd.DataFrame(results_list)

    # Calculate Averages
    overall_stats = df.mean(numeric_only=True)

    print("\n\n--- V3 Overall Average Statistics (Test Set) ---")
    summary_data = []
    for i in range(NUM_CLASSES):
        c_name = CLASS_MAP[i]
        summary_data.append({
            "Class": c_name,
            "Dice (↑)": overall_stats.get(f"{c_name}_Dice"),
            "IOU (↑)": overall_stats.get(f"{c_name}_IOU"),
            "HD95 (↓)": overall_stats.get(f"{c_name}_HD95"),
        })

    summary_df = pd.DataFrame(summary_data)
    print(summary_df.to_markdown(index=False, floatfmt=".4f"))

    # Optional: Save to CSV for comparison later
    summary_df.to_csv("/content/drive/MyDrive/Colab Notebooks/phenocyte_seg/phenocyte_seg/v3_results_summary.csv", index=False)
    print("\nSummary saved to v3_results_summary.csv")

if __name__ == "__main__":
    run_analysis()

Starting V3 Analysis...
GT Directory: /content/drive/MyDrive/Colab Notebooks/phenocyte_seg/phenocyte_seg/combined_data/masks/
Pred Directory: /content/drive/MyDrive/Colab Notebooks/phenocyte_seg/phenocyte_seg/outputs_v3/pred_masks_unsqueezed/test_set/


  0%|          | 0/52 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.metrics.utils get_mask_edges:always_return_as_numpy: Argument `always_return_as_numpy` has been deprecated since version 1.5.0. It will be removed in version 1.7.0. The option is removed and the return type will always be equal to the input type.
  warn_deprecated(argname, msg, warning_category)
  2%|▏         | 1/52 [00:00<00:06,  7.72it/s]/usr/local/lib/python3.12/dist-packages/monai/metrics/utils.py:327: UserWarning: the ground truth of class 3 is all 0, this may result in nan/inf distance.
  warnings.warn(
 25%|██▌       | 13/52 [00:01<00:03, 11.58it/s]/usr/local/lib/python3.12/dist-packages/monai/metrics/utils.py:327: UserWarning: the ground truth of class 4 is all 0, this may result in nan/inf distance.
  warnings.warn(
 29%|██▉       | 15/52 [00:01<00:03, 11.96it/s]/usr/local/lib/python3.12/dist-packages/monai/metrics/utils.py:332: UserWarning: t



--- V3 Overall Average Statistics (Test Set) ---
| Class      |   Dice (↑) |   IOU (↑) |   HD95 (↓) |
|:-----------|-----------:|----------:|-----------:|
| Background |     0.9942 |    0.9885 |     7.1458 |
| Stem       |     0.7149 |    0.5774 |     7.1714 |
| Leaf       |     0.8360 |    0.7231 |     7.7421 |
| Root       |     0.8103 |    0.6908 |    10.4944 |
| Seed       |     0.6350 |    0.5143 |    20.8207 |

Summary saved to v3_results_summary.csv
